In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
file = pd.read_csv('AS-SP500.csv')
df = pd.DataFrame(file)

In [ ]:
#obtener los nombres de los tickers
def get_tickers(df):
  df_tickers = df[df.columns[2]]
  tickers = pd.unique(pd.Series(df_tickers))
  return tickers

#crear los distintos CSV's por ticker
def create_csv_ticker(df, tickers):
  df_tickers = df[df.columns[2]]
  for ticker in tickers:
    new_df = df[df_tickers == ticker]
    new_df.to_csv(ticker+'.csv', encoding='utf-8', index=False)

#Parsear String a timestamp
def parse_time(x):
    datetime_object = datetime.strptime(x,'%H:%M:%S')
    return datetime_object 

#Obtener los indices y el numero de columnas faltantes
def get_index(df):
  news_rows = []
  for row in df.itertuples(index=True):
    index = row.Index
    date_1 = parse_time(df['timestamp'][index])
    if index < (len(df)-1):
      date_2 =  parse_time( df['timestamp'][index+1])
    else:
      date_2 =  parse_time( df['timestamp'][index])
    minutes = ((date_2 - date_1).seconds)/60
    if minutes > 1:
      news_rows.append({'index': index, 'num': minutes-1})
  return news_rows

#Funcion que agrega las filas faltante y crea un nuevo DF
def create_df(df, news_rows):
  indexs = []
  num_rows = []
  for key in news_rows:
    indexs.append(key['index'])
    num_rows.append(key['num'])

  nf = pd.DataFrame(columns=['date', 'timestamp', 'ticker', 'open',
                             'high', 'low', 'close', 'volume', 
                             'quantity', 'tradecount'])
  index = 0
  for row in df.itertuples(index=True):
    lista= [row.date, row.timestamp, row.ticker,
            row.open, row.high, row.low, row.close,
            row.volume, row.quantity, row.tradecount]
    
    nf.loc[index] = lista
    
    if row.Index in indexs:
        index_array = indexs.index(row.Index)
        index = index + 1
        
        key = 0
        while key < num_rows[index_array]:
          nf.loc[index] = lista
          key = key + 1
          index = index + 1
            
    index = index + 1
  nf.to_csv('new1.csv', encoding='utf-8', index=False)
  return nf